In [0]:
# from google.colab import files
# uploaded = files.upload()

Saving IH1801_20180102.csv to IH1801_20180102.csv


In [0]:
# import io
# import pandas as pd

In [0]:
# df2 = pd.read_csv(io.BytesIO(uploaded['IH1801_20180102.csv']))

In [0]:
# df2.head()

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# '1eD5klEOuUXzzwqavb3qoaCfHlYkogf0s' is in shared google drive link of the IH folder
# listed = drive.ListFile({'q': "title contains '.csv' and '1eD5klEOuUXzzwqavb3qoaCfHlYkogf0s' in parents"}).GetList()
# for file in listed:
#   print('title {}, id {}'.format(file['title'], file['id']))


In [0]:
# downloaded = drive.CreateFile({'id': '1pBgwamSnaGKQmJqadNdVDzSDDFZYdaag'}) 
# downloaded.GetContentFile('IH1801_20180117.csv')  
# df = pd.read_csv('IH1801_20180117.csv')

In [0]:
import pandas as pd
import numpy as np
import datetime as dt
import timeit
import os.path

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
IH_dir = '/content/drive/My Drive/DL_Project/IH/'
IF_dir = '/content/drive/My Drive/DL_Project/IF/'
IC_dir = '/content/drive/My Drive/DL_Project/IC/'

In [0]:
front_month_map = {1: '1802', 2: '1803', 3: '1804', 4: '1805', 5: '1806', 6: '1807', 
           7: '1808', 8: '1809', 9: '1810', 10: '1811', 11: '1812', 12: '1901'}

In [0]:
Morning_Start = dt.timedelta(hours=9, minutes=30)
Morning_End = dt.timedelta(hours=11, minutes=30)
Afternoon_Start = dt.timedelta(hours=13)
Afternoon_End = dt.timedelta(hours=15)

In [0]:
def regularize(df, fill_last, fill_zero, reg_col = 'datetime', reg_str='0.5S'):
  df = df.set_index(reg_col)
  df = df.resample(reg_str).last()
  # ffill: propagate last valid observation forward to next valid
  df[fill_last] = df[fill_last].fillna(method='ffill')
  df[fill_zero] = df[fill_zero].fillna(0)
  return df

In [0]:
def add_features(df):
  
  # helper start:
  df['mid'] = 0.5*(df['bid1'] + df['ask1'])
  df['wmid'] = (df['bid1']*df['askv1'] + df['ask1']*df['bidv1']) / (df['askv1'] + df['bidv1'])
  
  df['trade_dir'] = 0 # approximation
  df.loc[((df['mid'] > df['mid'].shift(1)) | 
      (df['last'] >= df['ask1'].shift(1)) ) & (df['volume'] > 0), 'trade_dir'] = 1
  df.loc[((df['mid'] < df['mid'].shift(1)) | 
      (df['last'] <= df['bid1'].shift(1)) ) & (df['volume'] > 0), 'trade_dir'] = -1
  df['signed_volume'] = df['trade_dir'] * df['volume']
  # df['price_volume'] = df['volume'] * df['last']
  # df['price_volume'] = df['price_volume'].replace(to_replace=0, method='ffill')
  
  df['bid_bsize'] = df['bid1'] * df['bidv1']
  df['ask_asize'] = df['ask1'] * df['askv1']
  
  df['tick_up'] = 0
  df.loc[df['mid'] > df['mid'].shift(1), 'tick_up'] = 1
  
  df['tick_down'] = 0
  df.loc[df['mid'] < df['mid'].shift(1), 'tick_down'] = 1
  
  # helper end ---------------------------------------------------------------
  
  df['wmid_mid'] = df['wmid'] - df['mid']
  df['wmid_last'] = df['wmid'] - df['last']
  
  df['mid_lag_1tick'] = df['mid'] - df['mid'].shift(1)
  df['mid_lag_01s'] = df['mid'] - df['mid'].shift(1*2)
  df['mid_lag_05s'] = df['mid'] - df['mid'].shift(5*2)
  df['mid_lag_10s'] = df['mid'] - df['mid'].shift(10*2)
  df['mid_lag_30s'] = df['mid'] - df['mid'].shift(30*2)
  df['mid_lag_01m'] = df['mid'] - df['mid'].shift(1*60*2)
  df['mid_lag_05m'] = df['mid'] - df['mid'].shift(5*60*2)
  df['mid_lag_10m'] = df['mid'] - df['mid'].shift(10*60*2)
  # df['mid_lag_01s_lag1'] = df['mid'].shift(1*2) - df['mid'].shift(2*1*2)
  
  df['wmid_ma_05m'] = df['wmid'] - df['wmid'].rolling(5*60*2).mean()
  df['wmid_max_05m'] = df['wmid'] - df['wmid'].rolling(5*60*2).max()
  df['wmid_min_05m'] = df['wmid'] - df['wmid'].rolling(5*60*2).min()
  df['wmid_ma_10m'] = df['wmid'] - df['wmid'].rolling(10*60*2).mean()
  df['wmid_max_10m'] = df['wmid'] - df['wmid'].rolling(10*60*2).max()
  df['wmid_min_10m'] = df['wmid'] - df['wmid'].rolling(10*60*2).min()
  
  df['wmid_bidask_10m'] = df['wmid'] - ((df['bid_bsize'].rolling(10*60*2).mean() * df['askv1'].rolling(10*60*2).mean() + 
                                           df['ask_asize'].rolling(10*60*2).mean() * df['bidv1'].rolling(10*60*2).mean()) /
                                          (df['askv1'].rolling(10*60*2).mean() + df['bidv1'].rolling(10*60*2).mean()))
  df['wmid_bidask_01m'] = df['wmid'] - ((df['bid_bsize'].rolling(1*60*2).mean() * df['askv1'].rolling(1*60*2).mean() + 
                                           df['ask_asize'].rolling(1*60*2).mean() * df['bidv1'].rolling(1*60*2).mean()) /
                                          (df['askv1'].rolling(1*60*2).mean() + df['bidv1'].rolling(1*60*2).mean())) 
  
  df['total_volume_10s'] = df['volume'].rolling(10*2).sum()
  df['signed_volume_10s'] = df['signed_volume'].rolling(10*2).sum()
  df['signed_tick_10s'] = df['tick_up'].rolling(10*2).sum() - df['tick_down'].rolling(10*2).sum()

  df['total_volume_01m'] = df['volume'].rolling(60*2).sum()
  df['signed_volume_01m'] = df['signed_volume'].rolling(60*2).sum()
  df['signed_tick_01m'] = df['tick_up'].rolling(60*2).sum() - df['tick_down'].rolling(60*2).sum()
  
  df['total_volume_10m'] = df['volume'].rolling(10*60*2).sum()
  df['signed_volume_10m'] = df['signed_volume'].rolling(10*60*2).sum()
  df['signed_tick_10m'] = df['tick_up'].rolling(10*60*2).sum() - df['tick_down'].rolling(10*60*2).sum()
  
  
  # clean up helper columns:
  del df['bid_bsize']
  del df['ask_asize']
  del df['signed_volume']
  del df['tick_up']
  del df['tick_down']

  return df

In [0]:
def add_features_other_assets(df, ticker):
  
  # helper start:
  df['mid'] = 0.5*(df['bid1'] + df['ask1'])
  # helper end ---------------------------------------------------------------
  
  df['mid_lag_01s'] = df['mid'] - df['mid'].shift(1*2) 
  df['mid_lag_05s'] = df['mid'] - df['mid'].shift(5*2)  
  df['mid_lag_30s'] = df['mid'] - df['mid'].shift(30*2)  
  df['mid_lag_01m'] = df['mid'] - df['mid'].shift(1*60*2)
  df['mid_lag_05m'] = df['mid'] - df['mid'].shift(5*60*2)
  df['mid_lag_10m'] = df['mid'] - df['mid'].shift(10*60*2)

  df['total_volume_10s'] = df['volume'].rolling(10*2).sum()
  df['total_volume_01m'] = df['volume'].rolling(60*2).sum()
  df['total_volume_10m'] = df['volume'].rolling(10*60*2).sum()
  
  return df

In [0]:
def add_y(df):
  df['mid_1tick'] = df['mid'].shift(-1) - df['mid']  # future - current
  df['mid_01s'] = df['mid'].shift(-1*2) - df['mid']
  df['mid_05s'] = df['mid'].shift(-5*2) - df['mid']
  df['mid_10s'] = df['mid'].shift(-10*2) - df['mid']
  df['mid_30s'] = df['mid'].shift(-30*2) - df['mid']
  df['mid_01m'] = df['mid'].shift(-1*60*2) - df['mid']
  df['mid_05m'] = df['mid'].shift(-5*60*2) - df['mid']
  df['mid_10m'] = df['mid'].shift(-10*60*2) - df['mid']
  df['mid_15m'] = df['mid'].shift(-15*60*2) - df['mid']
  
  df['wmid_1tick'] = df['wmid'].shift(-1) - df['wmid']
  df['wmid_01s'] = df['wmid'].shift(-1*2) - df['wmid']
  df['wmid_05s'] = df['wmid'].shift(-5*2) - df['wmid']
  df['wmid_10s'] = df['wmid'].shift(-10*2) - df['wmid']
  df['wmid_30s'] = df['wmid'].shift(-30*2) - df['wmid']
  df['wmid_01m'] = df['wmid'].shift(-1*60*2) - df['wmid']
  df['wmid_05m'] = df['wmid'].shift(-5*60*2) - df['wmid']
  df['wmid_10m'] = df['wmid'].shift(-10*60*2) - df['wmid']
  df['wmid_15m'] = df['wmid'].shift(-15*60*2) - df['wmid']
  
  return df

In [0]:
def build_one_day_IH(df, other_assets, morning_session_start, morning_session_end,
                     afternoon_session_start, afternoon_session_end):
  df.rename(columns = {' instrument': 'instrument',
                     ' datetime': 'datetime',
                     ' last': 'last',
                     ' opi': 'opi',
                     ' turnover': 'turnover',
                     ' volume': 'volume',
                     ' bid1': 'bid1',
                     ' ask1': 'ask1',
                     ' bidv1': 'bidv1',
                     ' askv1': 'askv1'}, inplace = True)
  
  df = df[['datetime', 'last', 'opi', 'turnover', 'volume', 'bid1', 'ask1', 'bidv1', 'askv1']]
  fill_last_cols = ['last', 'opi', 'bid1', 'ask1', 'bidv1', 'askv1']
  fill_zero_cols = ['turnover', 'volume']

  df['datetime'] = pd.to_datetime(df['datetime'])

  df_am = df[(df['datetime'] >= morning_session_start) & 
             (df['datetime'] <= morning_session_end)]
  df_pm = df[(df['datetime'] >= afternoon_session_start) & 
             (df['datetime'] <= afternoon_session_end)]
  
  df_am = regularize(df_am, fill_last_cols, fill_zero_cols)
  df_pm = regularize(df_pm, fill_last_cols, fill_zero_cols)
  
  df_am = add_features(df_am)
  df_pm = add_features(df_pm)
  df_am = add_y(df_am)
  df_pm = add_y(df_pm)
  
  # expand columns
  for other_asset in other_assets:
    df_am = pd.merge(df_am, other_asset['am'], how='left', left_index=True, right_index=True)
    df_pm = pd.merge(df_pm, other_asset['pm'], how='left', left_index=True, right_index=True)
  
  # merge rows
  df = pd.concat([df_am, df_pm])
  return df
  

In [0]:
def build_one_day_other_asset(df, ticker, morning_session_start, morning_session_end,
                                  afternoon_session_start, afternoon_session_end):
  df.rename(columns = {' instrument': 'instrument',
                     ' datetime': 'datetime',
                     ' last': 'last',
                     ' opi': 'opi',
                     ' turnover': 'turnover',
                     ' volume': 'volume',
                     ' bid1': 'bid1',
                     ' ask1': 'ask1',
                     ' bidv1': 'bidv1',
                     ' askv1': 'askv1'}, inplace = True)
  
  df = df[['datetime', 'last', 'opi', 'turnover', 'volume', 'bid1', 'ask1', 'bidv1', 'askv1']]
  fill_last_cols = ['last', 'opi', 'bid1', 'ask1', 'bidv1', 'askv1']
  fill_zero_cols = ['turnover', 'volume']

  df['datetime'] = pd.to_datetime(df['datetime'])

  df_am = df[(df['datetime'] >= morning_session_start) & 
             (df['datetime'] <= morning_session_end)]
  df_pm = df[(df['datetime'] >= afternoon_session_start) & 
             (df['datetime'] <= afternoon_session_end)]
  
  df_am = regularize(df_am, fill_last_cols, fill_zero_cols)
  df_pm = regularize(df_pm, fill_last_cols, fill_zero_cols)
  
  df_am = add_features_other_assets(df_am, ticker)
  df_pm = add_features_other_assets(df_pm, ticker)
  
  df_am.columns = [ticker + '_' + c for c in df_am.columns]
  df_pm.columns = [ticker + '_' + c for c in df_pm.columns]
  
  return {'am': df_am, 'pm': df_pm}  

In [30]:
start_time = timeit.default_timer()

for trade_date in pd.date_range('20180101', '20181231'):
  contract = 'IH' + front_month_map[trade_date.month] + '_' + trade_date.strftime('%Y%m%d')
  contract_if = 'IF' + front_month_map[trade_date.month] + '_' + trade_date.strftime('%Y%m%d')
  contract_ic = 'IC' + front_month_map[trade_date.month] + '_' + trade_date.strftime('%Y%m%d')
  
  if not os.path.exists(IH_dir + contract + '.csv'):
    # print(IH_dir + contract + '.csv', ' not found')
    continue
  
  print('Processing', trade_date.date(), ' Contract:', contract)
  
  IH = pd.read_csv(IH_dir + contract + '.csv')
  IF = pd.read_csv(IF_dir + contract_if + '.csv')
  IC = pd.read_csv(IC_dir + contract_ic + '.csv')
  
  morning_start = trade_date + Morning_Start
  morning_end = trade_date + Morning_End
  afternoon_start = trade_date + Afternoon_Start
  afternoon_end = trade_date + Afternoon_End
  
   
  IF = build_one_day_other_asset(IF, 'IF', morning_start, morning_end,
                  afternoon_start, afternoon_end)
  IC = build_one_day_other_asset(IC, 'IC', morning_start, morning_end,
                   afternoon_start, afternoon_end)
  
  
  IH = build_one_day_IH(IH, [IF, IC], morning_start, morning_end,
              afternoon_start, afternoon_end)
  
  IH_dropna = IH.dropna()
  
  # IH.to_csv('/content/drive/My Drive/DL_Project/input_df_cross_assets_v2/raw_input_' + 
  #           trade_date.strftime('%Y%m%d') + '.csv.gz', compression='gzip')
  IH_dropna.to_csv('/content/drive/My Drive/DL_Project/input_df_cross_assets_v2/input_' + 
           trade_date.strftime('%Y%m%d') + '.csv.gz', compression='gzip')

print('Time took: ', timeit.default_timer() - start_time)

Processing 2018-01-02  Contract: IH1802_20180102


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Processing 2018-01-03  Contract: IH1802_20180103


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Processing 2018-01-04  Contract: IH1802_20180104
Processing 2018-01-05  Contract: IH1802_20180105
Processing 2018-01-08  Contract: IH1802_20180108
Processing 2018-01-09  Contract: IH1802_20180109
Processing 2018-01-10  Contract: IH1802_20180110
Processing 2018-01-11  Contract: IH1802_20180111
Processing 2018-01-12  Contract: IH1802_20180112
Processing 2018-01-15  Contract: IH1802_20180115
Processing 2018-01-16  Contract: IH1802_20180116
Processing 2018-01-17  Contract: IH1802_20180117
Processing 2018-01-18  Contract: IH1802_20180118
Processing 2018-01-19  Contract: IH1802_20180119
Processing 2018-01-22  Contract: IH1802_20180122
Processing 2018-01-23  Contract: IH1802_20180123
Processing 2018-01-24  Contract: IH1802_20180124
Processing 2018-01-25  Contract: IH1802_20180125
Processing 2018-01-26  Contract: IH1802_20180126
Processing 2018-01-29  Contract: IH1802_20180129
Processing 2018-01-30  Contract: IH1802_20180130
Processing 2018-01-31  Contract: IH1802_20180131
Processing 2018-02-0